## 1. Installation des librairies nécessaires

In [ ]:
! pip install transformers

In [ ]:
! pip install datasets

In [ ]:
! pip install evaluate

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
import evaluate
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch

## 2. Récupération et préparation des données

In [ ]:
film_df = pd.read_csv("allocine_genres_train.csv", sep=",")

In [ ]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(film_df.genre.unique())
label2id = {class_names[i]:i for i in range(len(class_names))}
id2label = {i:class_names[i] for i in range(len(class_names))}

In [ ]:
# Paramètres

batch_size = 8
# Proportion des données qui sera utilisée
scale = 0.2

In [ ]:
data_df = pd.DataFrame()
# Le texte décrivant chaque film est composé des colonnes titre et synopsis
data_df['text'] = film_df.titre + ' ' + film_df.synopsis
# La classe cible est le genre sous forme d'identifiant numérique
data_df['label'] = film_df.genre.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
province_features = Features({'text': Value('string'), 
                              'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=province_features)
# Découpage en train et test
data = data.train_test_split(test_size=0.2, shuffle=True, seed=12)

## 3. Tokénisation des données

Nous allons utiliser xlm-roberta-base (pour le français).


In [ ]:
model_ckpt = "xlm-roberta-base"
# Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
# Tokénisation des 2 premières instances
preprocess_function(data['train'][:2])

In [ ]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=None)

In [ ]:
# Affichage des tokens.
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['train'][0]['input_ids'])

## 4. Préparation de l'évaluation

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    return acc

## 5. Entraînement par affinage

On commence par charger le modèle pré-entraîné

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=len(class_names), id2label=id2label, label2id=label2id
).to(device)

In [ ]:

training_args = TrainingArguments(
    output_dir=f"{model_ckpt}-finetuned-wine",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## 6. Analyse des résultats

In [ ]:
# Prédictions pour les données de test
preds_output = trainer.predict(tokenized_data['test'])

In [ ]:
preds_output

In [ ]:
preds_output.metrics

---
Nous allons également afficher la matrice de confusion.





In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
y_valid = tokenized_data['test']['label']

In [ ]:
labels = tokenized_data['test'].features['label'].names

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                  display_labels=labels_for_fig)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    
plot_confusion_matrix(y_preds, y_valid, labels)

Enfin, nous allons analyser les erreurs de classification. Pour cela, nous allons trier les instances par perte décroissante.

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Fonction qui retourne la perte (entropie croisée) et la classe prédite
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), 
                             reduction="none")
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
# Conversion des données au bon format
tokenized_data.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Calcul des valeurs de perte
tokenized_data["test"] = tokenized_data["test"].map(
    forward_pass_with_label, batched=True, batch_size=64)

In [ ]:
# Création d'un DataFrame avec les textes, les pertes les classe (prédites et attendues)

def label_int2str(row):
    return tokenized_data["train"].features["label"].int2str(row)

tokenized_data.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = tokenized_data["test"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [ ]:
# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1)
# Affichage des 10 premières instances triées par perte décroissante
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
# Affichage des 10 premières instances triées par perte croissante
# Cela permet de voir les instances pour lesquelles les prédictions sont les plus certaines
df_test.sort_values("loss", ascending=True).head(10)

On fait désormais la prédiction sur le fichier test.csv car c'est la méthode qui a la meilleur précision

In [ ]:
film_df = pd.read_csv("allocine_genres_test.csv", sep=",")

In [ ]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(film_df.genre.unique())
label2id = {class_names[i]:i for i in range(len(class_names))}
id2label = {i:class_names[i] for i in range(len(class_names))}
data_df = pd.DataFrame()
# Le texte décrivant chaque film est composé des colonnes titre et synopsis
data_df['text'] = film_df.titre + ' ' + film_df.synopsis
# La classe cible est le genre sous forme d'identifiant numérique

# on laisse data_df['label] car cela n'est pas utilisé dans la prédiction du genre et
# cela permet de voir la précision de notre prédiction.
# Mais normalement, c'est ce qu'on cherche sur ce jeu de données, donc nous ne sommes pas censé
# savoir au final la précision sur le jeu (cela permet de voir si l'algo fonctionne même sans validation croisée)
# Il est possible de l'enlever, mais il faut faire quelque modification dans la suite du code
data_df['label'] = film_df.genre.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
province_features = Features({'text': Value('string'), 
                              'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=province_features)

## 7. Prédiction sur le jeu de test

In [ ]:
tokenized_data = data.map(preprocess_function, batched=False, batch_size=None)
preds_output = trainer.predict(tokenized_data)
preds_output.metrics


In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)
#fonction qui permet de convertir les id des genre predit en leur correspondance en texte.
def label_int2str(row):
    return tokenized_data.features["label"].int2str(row)

In [ ]:
pred_label = []
for row in y_preds:
  pred_label.append(label_int2str(int(row)))

In [ ]:
print(pred_label)

Création du csv enrichi

In [ ]:
# 1. Ajouter une nouvelle colonne à film_df
film_new_df = film_df.assign(genre_predit=np.nan)

# 2. Copier les valeurs de y_pred dans la colonne nouvellement créée
film_new_df['genre_predit'] = pred_label

# 3. Enregistrer le DataFrame mis à jour dans un nouveau fichier CSV
film_new_df.to_csv('data/allocine_genres_test_enrichi.csv', index=False)